In [1]:
import requests
import os
import pandas as pd
import json
from tqdm import tqdm
import datetime

In [ ]:

path = '/Users/apple/PROJECT/Code_4_10k/sp500_total_constituents.csv'

try:
    df = pd.read_csv(path, encoding = 'utf-8')
    cik = df['CIK'].drop_duplicates().tolist()
    ticker = df['Symbol'].tolist()
    cik_ticker = dict(zip(cik, ticker))
except UnicodeDecodeError:
    df = pd.read_csv(path, encoding = 'ISO-8859-1')
    cik = df['CIK'].drop_duplicates().tolist()
    ticker = df['Symbol'].tolist()
    cik_ticker = dict(zip(cik, ticker))

import requests
import os
from tqdm import tqdm

total_len = 0
valid = 0
save_folder = "SP500-historical-prices_2012-2024"
for ticker in tqdm(cik_ticker.values()):
    ticker = ticker.lower()
    file_name = f'{ticker}.json'
    file_path = os.path.join(save_folder, file_name)
    if os.path.exists(file_path):
        print(f"Skipping {ticker}: {file_path} already exists.")
        continue
    url = "https://seeking-alpha.p.rapidapi.com/analysis/v2/list"

    querystring = {"id":ticker,"until":"1704067200","since":"1136073600","size":"40","number":"1"}

    headers = {
        "x-rapidapi-key": "13f98f0478msha8ec97b6a805b1fp174175jsn98c8458d4397",
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=querystring).json()
		# response.raise_for_status()  # Raise HTTPError for bad responses (4xx and 5xx)
    
		# if 'application/json' in response.headers.get('Content-Type', ''):
		# 	data = response.json()  # Parse JSON if content is JSON
		# 	print(data)
		# else:
		# 	print("Response is not JSON")
		# 	print(response.text)  # Debug raw response
    except:
        continue

    total_len += len(response['data'])
    valid += 1

    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    with open(f'SP500-historical-prices_2012-2024/{ticker}.json', 'w') as json_file:
        json.dump(response, json_file)

print("The number of firm retrieved:", total_len)


  0%|          | 0/549 [00:00<?, ?it/s]

Skipping qcom: SP500-historical-prices_2012-2024/qcom.json already exists.
Skipping itw: SP500-historical-prices_2012-2024/itw.json already exists.
Skipping d: SP500-historical-prices_2012-2024/d.json already exists.
Skipping ati: SP500-historical-prices_2012-2024/ati.json already exists.


  1%|          | 5/549 [00:02<04:13,  2.15it/s]

Skipping wmb: SP500-historical-prices_2012-2024/wmb.json already exists.
Skipping adsk: SP500-historical-prices_2012-2024/adsk.json already exists.
Skipping emr: SP500-historical-prices_2012-2024/emr.json already exists.
Skipping kbh: SP500-historical-prices_2012-2024/kbh.json already exists.
Skipping cost: SP500-historical-prices_2012-2024/cost.json already exists.


 11%|█▏        | 63/549 [03:48<30:57,  3.82s/it]  

In [3]:

year2unixTime = {}
for year in range(2024, 2006, -1):
    current_year_timestamp = int(datetime.datetime(year, 1, 1).timestamp())
    year2unixTime[year] = current_year_timestamp
    
year2unixTime

{2024: 1704067200,
 2023: 1672531200,
 2022: 1640995200,
 2021: 1609459200,
 2020: 1577836800,
 2019: 1546300800,
 2018: 1514764800,
 2017: 1483228800,
 2016: 1451606400,
 2015: 1420070400,
 2014: 1388534400,
 2013: 1356998400,
 2012: 1325376000,
 2011: 1293840000,
 2010: 1262304000,
 2009: 1230768000,
 2008: 1199145600,
 2007: 1167609600}

In [66]:
import requests
import os
import pandas as pd
import json
from tqdm import tqdm
import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import atexit
from threading import Lock
from collections import defaultdict

path = '/Users/apple/PROJECT/Code_4_10k/sp500_total_constituents.csv'

try:
    df = pd.read_csv(path, encoding='utf-8')
    cik = df['CIK'].drop_duplicates().tolist()
    ticker = df['Symbol'].tolist()
    cik_ticker = dict(zip(cik, ticker))
except UnicodeDecodeError:
    df = pd.read_csv(path, encoding='ISO-8859-1')
    cik = df['CIK'].drop_duplicates().tolist()
    ticker = df['Symbol'].tolist()
    cik_ticker = dict(zip(cik, ticker))

total_len = 0

valid = 0
total_requests = 0
save_folder = "analysis_reports"
id_folder = "analysis_report_ids"
year_until = 2024
year_since = 2006

# Rate limiting variables
rate_limit = 5
request_counter = 0
last_request_time = time.time()
lock = Lock()

def fetch_ids_for_ticekr(ticker):
    year_id2title = defaultdict(dict)
    total_numIds = 0
    year_ids = 0
    ticker = ticker.lower()
    ticker_id_folder = os.path.join(id_folder, ticker)
    if not os.path.exists(ticker_id_folder):
        os.makedirs(ticker_id_folder)
        
    for year in range(year_until, year_since, -1):
        id2title = defaultdict(lambda: 'title')
        year_file_path = os.path.join(ticker_id_folder, f"{year-1}.json")
        if not os.path.exists(year_file_path):
            continue
        with open(year_file_path, 'r') as json_file:
            json_data = json.load(json_file)
            if 'data' in json_data and json_data['data']:                
                for i in range(len(json_data['data'])):
                    id = json_data['data'][i]['id']
                    title = json_data['data'][i]['attributes']['title']
                    id2title[id] = title
        year_id2title[year] = id2title
        year_ids = len(id2title)
        total_numIds += year_ids

    return year_id2title, total_numIds
            

In [67]:
result, ids = fetch_analysis_for_ticekr('cost')
print(result)
print(len(result[2024]))
print(len(result[2023]))
print(ids)

defaultdict(<class 'dict'>, {2024: defaultdict(<function fetch_analysis_for_ticekr.<locals>.<lambda> at 0x7f77bb8bf4c0>, {'4660494': "Costco's Growth Will Persist, Even If The U.S. Enters A Recession", '4659603': 'Costco: Not As Overvalued As Many Think', '4659401': 'Costco Vs. Walmart: Several Reasons Make For A Clear Winner', '4658761': 'Costco: This Is Where The Bold Will Turn Cautious (Rating Downgrade)', '4658361': 'Costco: A Buy At New 52-Week Highs And After Announcing $15 Special Dividend?', '4658162': 'Costco Q1: Membership Model Muscles Through, But Can It Maintain Momentum?', '4657176': "Costco's Earnings: A Few Thoughts On The Special Dividend And The Fee Hike", '4657153': "Note To BJ's Wholesale Club Investors: There's Only One Costco", '4655484': 'Costco: Extremely Attractive Business But Unappealing Valuation', '4654687': 'Costco: Same Membership Fee For More Than Six Years; It Is Time Now', '4653030': "Costco: I Can't Defend A 40x P/E", '4649600': 'Costco: The Retailer 